## Import All the Stuff

In [182]:
import sys
import os
from crewai import Process, Crew, Agent, Task
from crewai_tools import BaseTool, SerperDevTool
import requests
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

## Hardcode the local LLM (for development purposes, otherwise we'll use the .env)

In [183]:
load_dotenv()

True

## Choosing an internet search api instead

In [184]:
search_tool = SerperDevTool()

# ---------------------- Agents Definition ----------------------


### 1. Requirements Gathering Agent

In [185]:
requirements_agent = Agent(
    role="Requirement Gathering Agent",
    goal="Gather and structure user requirements, ensuring all essential metadata fields are captured in a JSON format.",
    backstory=(
"You are an expert AWS consultant skilled in structured requirement gathering. "
              "Your job is to conduct detailed interviews with users to extract clear and actionable requirements. "
              "You categorize user inputs across key attributes such as cost profile, scalability, security, "
              "performance, and compliance needs. "
              "Your structured output enables other agents to accurately retrieve AWS documentation and generate solutions. "
              "You always clarify ambiguous responses and ensure completeness before finalizing requirements."
),
    allow_delegation=False,
    
    verbose=True,
)


### 2. Knowledge Retrieval Agent

In [186]:
knowledge_agent = Agent(
    role="Knowledge Agent",
    goal="Retrieve and process AWS documentation to provide up-to-date and reliable information.",
    backstory=(
"You are an AI-powered AWS documentation specialist and a prompt engineering expert. " 
"Your prompt engineering skills enable you to craft optimal queries to pass to the LLM-based internet search tool, from structured user requirements, " 
"ensuring that your search results are precise, comprehensive, and contextually relevant."
"Your expertise lies in retrieving, analyzing, and summarizing AWS documentation, guides, decision guides, implementation guides, whitepapers, case studies, best practices, tutorials, and reference architectures and use cases to generate a base solution tailored to the user’s needs"

),
    allow_delegation=False,
    
    verbose=True,
)


### 3. Implementation Examples Agent

In [187]:
implementation_agent = Agent( 
    role="Implementation Example Agent", 
    goal="Search real-world AWS implementation examples on GitHub to refine and support the generated architecture solution.", 
    backstory=(
    "You specialize in finding practical AWS implementation examples by searching GitHub repositories. " 
    "Your task is to provide real-world code examples and reference implementations that align with the proposed solution. " 
    "This helps validate the feasibility of the recommendations and offers tangible examples for further refinement."
    ), 
    allow_delegation=True, 
    
    verbose=True, 
)


### 4. QA and Validation Agent


In [188]:
qa_validation_agent = Agent( 
    role="QA and Validation Agent", 
    goal="Ensure the AWS architecture recommendation is comprehensive, accurate, and aligned with AWS best practices, including service details, quotas, and integration requirements.", 
    backstory=(
    "You are a meticulous QA specialist with deep expertise in AWS documentation and best practices. " 
    "Your role is to review the architecture recommendation generated by the Solution Architect Agent. " 
    "You must verify that the solution addresses all aspects of the user requirements (cost, scalability, security, performance, and compliance) " 
    "and is fully supported by up-to-date AWS service documentation. " 
    "Ensure that every recommendation is backed by concrete references, that all assumptions are clearly stated, " 
    "and that there are no gaps in the technical details (such as quotas or integration specifics). " 
    "If any part of the solution is incomplete or contradictory, delegate the refinement back to the appropriate agent." 
    ), 
    allow_delegation=True, 
    
    verbose=True, 
)


### 5. Solution Architect Agent

In [189]:
solution_agent = Agent(
    role="Solution Agent",
    goal="Generate architecturally sound AWS solutions tailored to user needs.",
    backstory=(
"You are a highly experienced AWS Solutions Architect, skilled in designing scalable, cost-effective, and secure cloud architectures. "
              "Using structured user requirements and retrieved AWS documentation, you craft tailored AWS architecture recommendations "
              "that align with best practices, compliance, and performance needs."
    ),
    allow_delegation=False,
    
    verbose=True,
)


#  ---------------------- Tasks Definition ----------------------


### Task 1: Requirements Gathering

In [190]:
task_requirements = Task(
    description=(
        "Conduct structured interviews with the user to gather key requirements for the AWS solution. "
        "Ask targeted questions about cost, scalability, ease of implementation, security, and other relevant factors. "
        "Ensure responses align with predefined metadata categories. "
        "Clarify ambiguous answers and document the final requirements in JSON format with the following structure:\n\n"

        "```json\n"
        "{"
        '  "cost_profile": "High Cost | Medium Cost | Low Cost | Cost-Optimized",\n'
        '  "scalability": "High (millions of users) | Medium | Low (small-scale)",\n'
        '  "implementation": {\n'
        '      "ease": "Easy | Moderate | Difficult",\n'
        '      "integration": "Easy | Moderate | Difficult",\n'
        '      "expertise": "Beginner | Intermediate | Advanced"\n'
        '  },\n'
        '  "use_case": ["Video Streaming", "Data Analytics", "IoT", "Web Applications", "Machine Learning", "Databases", "Storage", "Networking", "Security", "DevOps", "Hybrid Cloud", "Serverless"],\n'
        '  "security_tier": "Basic | Enterprise",\n'
        '  "performance": "High Performance | Moderate | Low Performance",\n'
        '  "availability": "High Availability (multi-region) | Moderate | Low Availability",\n'
        '  "compliance": ["GDPR", "HIPAA", "PCI-DSS", "None"],\n'
        '  "implementation_time": "Short (days) | Medium (weeks) | Long (months+)"\n'
        "}\n"
        "```\n\n"

        "Ensure the user understands and confirms their selections before finalizing the output. "
        "This JSON will serve as the foundation for knowledge retrieval and solution generation."
    ),
    human_input=True,
    expected_output="A structured JSON object with categorized user requirements.",
    
    agent=requirements_agent
)

### Task 2: Knowledge Retrieval


In [191]:
task_knowledge_retrieval = Task(
    description=(
"Retrieve relevant AWS documentation, guides, decision guides, implementation guides, whitepapers, case studies, best practices, tutorials, and reference architectures and use cases to generate a base solution tailored to the user’s needs based on user requirements."
"Use internet search to ensure high relevance."
"Leverage your prompt engineering expertise to craft optimal queries from the structured requirements. "
    ),
    expected_output="A distilled summary of AWS documentation, guides, whitepapers, use cases, and tutorials that are directly relevant to the user's needs, providing a comprehensive base response for solution design.", 
    
    agent=knowledge_agent,
    tools=[search_tool],
)


### Task 3: Real-World Implementations Search 


In [192]:
task_implementation_search = Task( 
    description=
    "Refine the base response (the distilled summary) by searching GitHub repositories for real-world AWS implementation examples relevant to the user's requirements"
    "Integrate these real-world examples to enhance and validate the AWS documentation summary, ensuring the solution is grounded in proven practices."
    "Summarize findings with links and code references.", 
    expected_output="A structured summary of real-world AWS implementations, including concrete examples and references.", 
    
    agent=implementation_agent,
    tools=[search_tool],
)


### Task 4: QA and Validation

In [193]:
task_validation = Task( 
    description=
    "Review the AWS architecture recommendation generated by the Solution Architect Agent. " 
    "Ensure that all user requirements are addressed: confirm that cost, scalability, security, performance, and compliance factors are properly covered. " 
    "Search AWS service documentation using the PerplexicaSearchTool to verify details such as service quotas, integration requirements, and best practices. " 
    "Check that every recommendation is supported with references and that assumptions are explicitly noted. " 
    "Provide a detailed validation report that outlines any discrepancies or areas for improvement, and, if necessary, delegate revisions back to the Solution Architect Agent.", 
    expected_output= 
    "A detailed validation report that includes:\n" 
    "- Confirmation that all user requirements are met by the proposed architecture.\n" 
    "- References and supporting evidence from AWS documentation for each recommendation.\n" 
    "- Identification of any discrepancies, missing details, or assumptions that need clarification.\n" 
    "- Actionable recommendations for refinements, if any issues are detected.",
    
    agent=qa_validation_agent, 
    tools=[search_tool], 
)


### Task 5: Solution Generation


In [194]:
task_solution_generation = Task(
    description="Generate a detailed AWS architecture recommendation based on user requirements and retrieved AWS documentation. "
                "Ensure the solution is optimized for cost, scalability, security, and compliance.",
    expected_output="A structured AWS architecture recommendation document.",
    
    agent=solution_agent
)

# -------------------------- Crew Assembly --------------------------


In [195]:
#from langchain.embeddings import OllamaEmbeddings
#from crewai.memory import ShortTermMemory
#embeddings = OllamaEmbeddings(model="nomic-embed-text")
#stm = ShortTermMemory(embeddings=embeddings)

In [196]:
crew = Crew(
    #agents=[ requirements_agent, knowledge_agent, implementation_agent, qa_validation_agent, solution_agent ], 
    #tasks=[ task_requirements, task_knowledge_retrieval, task_implementation_search, task_validation, task_solution_generation ],
    agents=[ knowledge_agent, implementation_agent, qa_validation_agent, solution_agent ], 
    tasks=[ task_knowledge_retrieval, task_implementation_search, task_validation, task_solution_generation ],
    process=Process.sequential,  # Ensures logical step-by-step execution
    
    verbose=2,  # Set verbosity level for detailed logs during execution
    #short_term_memory=stm,
    #memory=True
)


2025-03-29 14:04:47,857 - 14928 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# -------------------------- Run Crew --------------------------


In [197]:
user_requirements = {
    'cost_profile': "Cost-Optimized",
    'scalability': "Medium",
    'implementation': {
        'ease': "Easy",
        'integration': "Moderate",
        'expertise': "Intermediate"
    },
    'use_case': "AI-powered customer support chatbot",
    'security_tier': "Enterprise",
    'performance': "High Performance",
    'availability': "High Availability (multi-region)",
    'compliance': ["GDPR", "HIPAA"],
    'implementation_time': "Short (days)"
}


In [198]:
crew.kickoff(inputs=user_requirements)

 [DEBUG]: == Working Agent: Knowledge Agent
 [INFO]: == Starting Task: Retrieve relevant AWS documentation, guides, decision guides, implementation guides, whitepapers, case studies, best practices, tutorials, and reference architectures and use cases to generate a base solution tailored to the user’s needs based on user requirements.Use internet search to ensure high relevance.Leverage your prompt engineering expertise to craft optimal queries from the structured requirements. 


> Entering new CrewAgentExecutor chain...
Thought: I need to understand the user's requirements to craft optimal queries for searching AWS documentation. However, since the specific user requirements are not provided, I will have to create a general query that can fetch relevant information about AWS services, guides, and documentation.

Action: Search the internet
Action Input: {"search_query": "AWS documentation guides decision guides implementation guides whitepapers case studies best practices tutorials r

'**Detailed AWS Architecture Recommendation Document**\n\n**Introduction:**\nThis document provides a comprehensive AWS architecture recommendation tailored to optimize cost, scalability, security, and compliance. The proposed solution aligns with best practices and leverages AWS services to ensure a highly efficient and secure cloud environment.\n\n**Overview of Requirements:**\nThe user requires an AWS architecture that is optimized for cost, scalability, security, and compliance. The solution must adhere to best practices for optimizing AWS costs while maintaining performance.\n\n**Architecture Design:**\n\n1. **Region Selection:** Choose the most appropriate AWS region based on factors such as proximity to users, regulatory requirements, and availability of services.\n2. **Compute Services:**\n\t* Use Amazon EC2 instances with a mix of On-Demand, Reserved Instances (RIs), and Spot Instances to optimize costs.\n\t* Implement Auto Scaling to dynamically adjust instance capacity based